In [1]:
import pandas as pd
import numpy as np
import math
import matplotlib
import matplotlib.pyplot as plt
import sklearn.metrics as metrics
import itertools as itertools
from sklearn import ensemble
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn import tree
from copy import deepcopy
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from bs4 import BeautifulSoup
import requests
import time


In [2]:
#function to get all game ids for one team
def year_schedule(months, year):
    game_ids = []
    for m in months:
        url1 = "https://www.basketball-reference.com/leagues/NBA_"+ year + "_games-"
        req = requests.get(url1 + m + '.html')
        page = req.text
        soup = BeautifulSoup(page, 'html.parser')
        print(m, year)
        div = soup.find('div', {'id': 'div_schedule'})
        if div != None:
            table = div.find('table', {'id': 'schedule'})
            rows = table.find_all('tr')
            for row in rows:
                center_cells = row.find_all('td', 'center', {'data-stat': 'box_score_text'})
                for center_cell in center_cells:
                    if center_cell.get_text() == "Box Score":
                        link = center_cell.find_all('a')[0]
                        url = link.get('href')
                        game_id = url.split('/')[2]
                        game_ids.append(game_id)
    return game_ids

def to_pbp_url(game_id):
    return "https://www.basketball-reference.com/boxscores/pbp/" + game_id

In [3]:
months = ['october', 'november', 'december', 'january', 'february', 'march', 'april', 'may', 'june']

In [4]:
years = ['2009', '2010', '2011', '2012', '2013', '2016', '2017']
schedule = []
for year in years:
    schedule = schedule + year_schedule(months, year)
    time.sleep(2)
len(schedule)

october 2009
november 2009
december 2009
january 2009
february 2009
march 2009
april 2009
may 2009
june 2009
october 2010
november 2010
december 2010
january 2010
february 2010
march 2010
april 2010
may 2010
june 2010
october 2011
november 2011
december 2011
january 2011
february 2011
march 2011
april 2011
may 2011
june 2011
october 2012
november 2012
december 2012
january 2012
february 2012
march 2012
april 2012
may 2012
june 2012
october 2013
november 2013
december 2013
january 2013
february 2013
march 2013
april 2013
may 2013
june 2013
october 2016
november 2016
december 2016
january 2016
february 2016
march 2016
april 2016
may 2016
june 2016
october 2017
november 2017
december 2017
january 2017
february 2017
march 2017
april 2017
may 2017
june 2017


8951

In [5]:
schedule1 = [to_pbp_url(x) for x in schedule]

In [15]:
dataframes1 = []
#dataframes2 = []

#schedule2 = [to_pbp_url(x) for x in schedule2]
start = time.time()
for url in schedule1[7000:7300]:
    r = requests.get(url)
    if r.status_code != 404:
        pbp = pd.read_html(url,header=1)[0]
        #pbp.columns = ['time', 'awayevents','awaypts','score','homepts','homeevents']
        dataframes1.append(pbp)
    time.sleep(2)
print(time.time() - start)
    
# for url in schedule2:
#     r = requests.get(url)
#     if r.status_code != 404:
#         pbp = pd.read_html(url,header=1)[0]
#         #pbp.columns = ['time', 'awayevents','awaypts','score','homepts','homeevents']
#         dataframes2.append(pbp)
#     time.sleep(5)
dataframes1[0].head()

ConnectionAbortedError: [WinError 10053] An established connection was aborted by the software in your host machine

In [8]:
test1 = [i.copy() for i in dataframes1]
test2 = [i.copy() for i in dataframes1]
test3 = [i.copy() for i in dataframes1]
test4 = [i.copy() for i in dataframes1]

In [9]:
def time_to_int(t):
    if len(t) > 5:
        numbers = t.split(":")
        seconds = numbers[1].split(".")
        t_sec = (int(numbers[0])*60) + int(seconds[0])
    else:
        t_sec = None
    return t_sec

In [10]:
#function to format scraped data
#TO DO: add 3 pointers, fouls, and rebounds for each team
def format_frames(pbp):
    awayteam = pbp.columns[1]
    hometeam = pbp.columns[5]
    pbp.columns = ['time', 'awayevents','awaypts','score','homepts','homeevents']
    pbp['awayteam'] = awayteam
    pbp['hometeam'] = hometeam
    events = pbp['awayevents']
    events = events.fillna(pbp['homeevents'])
    pbp['event'] = events
    #possession with 1 indicating that the away team is making the play and 0 indicating the opposite
    pbp['possession'] = 1-pd.isnull(pbp['awayevents'])
    #first need to replace scores at the beginning of games and the rows that just mark the end of quarters 
    pbp['score'] = pbp['score'].replace(to_replace='Score',method='ffill')
    pbp['score'] = pbp['score'].fillna(method='ffill')
    pbp['score'] = pbp['score'].fillna(method='bfill')

    #then we split these to the away team scores and home team scores
    #awayscore,homescore = score
    #print([len(x.split('-')) for x in pbp['score']])
    pbp['score'] = [str(x) for x in pbp['score']]
    score = [x.split('-') for x in pbp['score']]
    awayscore,homescore = np.transpose(np.array(score))
    awayscore = [int(x) for x in awayscore]
    homescore = [int(x) for x in homescore]
    pbp['awayscore'] = awayscore
    pbp['homescore'] = homescore

    #now drop the redundant variables
    pbp = pbp.drop(['awayevents','awaypts','score','homepts','homeevents'], axis=1)
    pbp1 = pbp[pd.notnull(pbp['event'])]
    #adding quarter to dataframe
    quarter = []
    count2 = 1
    for x, y in zip(pbp1['event'], pbp1['time']):
        if 'End of' in x and '0:00.0' in y:
            count2 += 1;
        quarter.append(count2)

    pbp1['quarter'] = quarter
    pbp1['time'] = pbp1['time'].apply(time_to_int)
    #getting rid of time = null rows
    pbp2 = pbp1[pd.notnull(pbp1['time'])]
    total_time = []
    for x, y in zip(pbp2['quarter'], pbp2['time']):
        if y == 0:
            time = 720*(x)
            if time == 0:
                total_time.append(2880)
            else:
                total_time.append(time)
        else:
            total_time.append(2880 - ((5-x)*y))
    pbp2["total_time"] = total_time
    #getting rid of redundant time = 0 
    z_count = -1
    for time in pbp2['total_time']:
        if time == 0:
            z_count += 1
    pbp2 = pbp2.iloc[z_count:]
    away_final = np.max(pbp2['awayscore'])
    home_final = np.max(pbp2['homescore'])
    if away_final > home_final:
        pbp2['win'] = [1 for x in pbp2['possession']]
    else: 
        pbp2['win'] = [0 for x in pbp2['possession']]
    return pbp2

In [11]:
formatted_frames = list(map(format_frames, test3))
for i, dataframe in enumerate(formatted_frames):
    dataframe["game_id"] = i
season_df = pd.concat(formatted_frames, ignore_index = True)
season_df

C:\Users\charl\Anaconda3\lib\site-packages\ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\charl\Anaconda3\lib\site-packages\ipykernel_launcher.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\charl\Anaconda3\lib\site-packages\ipykernel_launcher.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexi

,time,awayteam,hometeam,event,possession,awayscore,homescore,quarter,total_time,win,game_id
0,720.0,Dallas,Phoenix,Jump ball: S. O'Neal vs. E. Dampier (S. Nash g...,1,0,2,1,0.0,1,0
1,708.0,Dallas,Phoenix,S. Nash makes 2-pt shot at rim (assist by G. H...,0,0,2,1,48.0,1,0
2,687.0,Dallas,Phoenix,D. Nowitzki misses 2-pt shot from 17 ft,1,0,2,1,132.0,1,0
3,684.0,Dallas,Phoenix,Defensive rebound by M. Barnes,0,0,2,1,144.0,1,0
4,675.0,Dallas,Phoenix,Turnover by M. Barnes (bad pass; steal by J. K...,0,0,2,1,180.0,1,0
5,675.0,Dallas,Phoenix,Personal foul by J. Richardson (drawn by J. Kidd),1,0,2,1,180.0,1,0
6,659.0,Dallas,Phoenix,A. Wright makes 2-pt shot from 17 ft,1,2,2,1,244.0,1,0
7,643.0,Dallas,Phoenix,S. Nash makes 2-pt shot at rim (assist by G. H...,0,2,4,1,308.0,1,0
8,635.0,Dallas,Phoenix,A. Wright misses 2-pt shot from 11 ft,1,2,4,1,340.0,1,0
9,633.0,Dallas,Phoenix,Defensive rebound by G. Hill,0,2,4,1,348.0,1,0


In [12]:
len(season_df['game_id'].unique())

149

In [13]:
season_reference = season_df[['game_id', 'time', 'quarter', 'event', 'total_time']].copy()
season_reference.head()
season_df = season_df.drop(['time', 'event', 'quarter'], axis=1)
season_df.head()
season_df.to_csv("project/Worker7_seasons.csv", sep = ',')
season_reference.to_csv("project/Worker7_season_reference.csv", sep = ',')

In [ ]:
def format_references(pbp):
    pbp1 = pbp.copy()
    for gid in pbp1['game_id'].unique():
        pbp2 = pbp1.loc[pbp1['game_id'] == gid]
        #loop to find shooting percentage
        h_array = []
        a_array = []
        h_makes = 0
        a_makes = 0
        home_count = 0
        away_count = 0
        for i, row in pbp2.iterrows():
            if "shot" in row['event'] and row['possession'] == 0:
                if "make" in row['event']:
                    h_makes += 1
                home_count += 1
            elif "shot" in row['event'] and row['possession'] == 1:
                if "make" in row['event']:
                    a_makes += 1
                away_count += 1
            if home_count == 0:
                h_array.append(0.0)
            else:
                h_array.append((h_makes/home_count))
            if away_count == 0:
                a_array.append(0.0)
            else:
                a_array.append((a_makes/away_count))
        pbp2['home_percentage'] = h_array
        pbp2['away_percentage'] = a_array
    pbp2 = pbp2.drop(['event', 'quarter', 'awayteam', 'hometeam'], axis = 1)
    return pbp2

In [ ]:
hope = []
for g, r in zip(frames, references):
    n_df = r
    n_df = n_df.drop(['Unnamed: 0'], axis = 1)
    terms = ['awayteam', 'hometeam', 'possession', 'awayscore', 'homescore', 'win']
    for term in terms:
        n_df[term] = g[term]
    hope.append(n_df)
    
hope[0].head()